## 0. Import modules

In [1]:
import os
import random
import sys
from datetime import datetime
from subprocess import Popen

import h5py
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, ConcatDataset, TensorDataset

try:
    import pytorch_lightning as pl
    from pytorch_lightning import seed_everything
    from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
    from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar, Callback
except ImportError:
    !pip install pytorch-lightning
    import pytorch_lightning as pl
    from pytorch_lightning import seed_everything
    from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
    from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar, Callback

try:
    import snntorch as snn
    from snntorch import surrogate, utils
except ImportError:
    !pip install snntorch
    import snntorch as snn
    from snntorch import surrogate, utils

from tqdm import tqdm

try:
    from codecarbon import EmissionsTracker
except ImportError:
    !pip install codecarbon
    from codecarbon import EmissionsTracker

## 1. Runtime environment detection and configuration

In [2]:
def setup_environment():
    """Setup paths and detect if running on Google Colab."""
    try:
        # Check if running on Google Colab
        from google.colab import drive

        drive.mount('/content/drive', force_remount=True)
        base_dir = '/content/TFM/SNN/'
        os.makedirs(f"{base_dir}CL", exist_ok=True)

        # Sync data from Google Drive
        sync_command = "rsync -avP /content/drive/MyDrive/TFM/SNN/data/ /content/TFM/SNN/data/"
        os.system(sync_command)

        is_colab = True
    except ImportError:
        # Fallback for local environment
        base_dir = '/home/scosta/TFM/'
        is_colab = False

    # Change to the base directory
    os.chdir(base_dir)

    # Print the current working directory
    print(f"Current working directory: {os.getcwd()}")

    return is_colab, base_dir


# Detect environment and set up paths
is_colab, base_dir = setup_environment()

Current working directory: /home/scosta/TFM


## 2. Custom CALLBACKS definition

In [3]:
# Custom progressbar to avoid text cascading bug
class MyProgressBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        return self._disable_tqdm_if_not_tty(super().init_validation_tqdm())

    def init_predict_tqdm(self):
        return self._disable_tqdm_if_not_tty(super().init_predict_tqdm())

    def init_test_tqdm(self):
        return self._disable_tqdm_if_not_tty(super().init_test_tqdm())

    @staticmethod
    def _disable_tqdm_if_not_tty(bar):
        if not sys.stdout.isatty():
            bar.disable = True
        return bar


# Codecarbon callback to keep track of CO2 emissions
class CodeCarbonCallback(Callback):
    def __init__(self, tracker, verbose=True):
        self.tracker = tracker
        self.verbose = verbose
        self.state = {"co2_emissions": 0}

    def on_fit_start(self, trainer, pl_module):
        self.tracker.start()

    def on_fit_end(self, trainer, pl_module):
        self.state["co2_emissions"] = self.tracker.stop()

    def on_train_epoch_end(self, *args, **kwargs):
        self.state["co2_emissions"] = self.tracker.flush()
        self.log("co2_emissions", self.state["co2_emissions"])

    def load_state_dict(self, state_dict):
        self.state.update(state_dict)

    def state_dict(self):
        return self.state.copy()

## 3. CL: Elastic Weight Consolidation (EWC) implementation

In [4]:
class EWC:
    """
    Elastic Weight Consolidation (EWC) for continual learning.
    This class computes and applies EWC regularization to prevent catastrophic forgetting.
    """

    DEFAULT_DEVICE = "cuda"

    def __init__(self, model):
        """
        Initialize the EWC instance.
        :param model: PyTorch model to be regularized.
        """
        self.model = model
        self.prev_params = {}
        self.fisher_info = {}

    def compute_fisher(self, dataloader, device=DEFAULT_DEVICE):
        """
        Compute the Fisher Information Matrix for the model's parameters.
        :param dataloader: DataLoader providing the dataset for Fisher computation.
        :param device: Device on which computation is performed ('cuda' or 'cpu').
        """
        if not dataloader:
            raise ValueError("DataLoader cannot be None.")

        self.model.eval()
        fisher_info_matrix = self._initialize_fisher_info(device)

        # Progress bar and Fisher computation loop
        for images, labels in tqdm(dataloader, desc="Computing Fisher Information"):
            images, labels = images.to(device), labels.to(device)
            self._accumulate_fisher_info(images, labels, fisher_info_matrix, device)

        # Normalize Fisher Information by dataset size
        self._normalize_fisher_info(fisher_info_matrix, len(dataloader))
        self.fisher_info = fisher_info_matrix

    def save_prev_params(self):
        """
        Save the model's parameters for EWC regularization.
        """
        self.prev_params = {
            parameter_name: param.clone().detach()
            for parameter_name, param in self.model.named_parameters()
            if param.requires_grad
        }

    def ewc_loss(self, device=DEFAULT_DEVICE):
        """
        Compute the EWC loss for regularization.
        :param device: Device on which loss computation is performed ('cuda' or 'cpu').
        :return: EWC regularization loss.
        """
        if not self.prev_params or not self.fisher_info:
            raise ValueError(
                "Previous parameters or Fisher information is missing. "
                "Ensure compute_fisher and save_prev_params are called."
            )

        regularization_loss = sum(
            self._compute_param_loss(parameter_name, param, device)
            for parameter_name, param in self.model.named_parameters()
            if parameter_name in self.prev_params
        )
        return regularization_loss

    def _initialize_fisher_info(self, device):
        """
        Initialize Fisher information matrix as zero tensors with the same shape
        as the model's parameters.
        :param device: Device on which tensors are created.
        :return: Initialized tensor dictionary.
        """
        return {
            parameter_name: torch.zeros_like(param, device=device)
            for parameter_name, param in self.model.named_parameters()
            if param.requires_grad
        }

    def _accumulate_fisher_info(self, images, labels, fisher_info_matrix, device):
        """
        Perform forward and backward passes to accumulate Fisher information.
        :param images: Input data batch.
        :param labels: Target labels batch.
        :param fisher_info_matrix: Dictionary accumulating Fisher information.
        :param device: Device on which tensors are computed.
        """
        self.model.zero_grad()
        predictions = self.model(images)

        # Adjust labels to match loss function expectations
        loss = self.model.loss_fn(predictions, labels.unsqueeze(1))
        loss.backward()

        for parameter_name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None:
                fisher_info_matrix[parameter_name] += param.grad.detach() ** 2

    def _normalize_fisher_info(self, fisher_info_matrix, dataset_size):
        """
        Normalize Fisher information matrix by dataset size.
        :param fisher_info_matrix: Dictionary containing Fisher information.
        :param dataset_size: Size of the dataset.
        """
        for parameter_name in fisher_info_matrix:
            fisher_info_matrix[parameter_name] /= dataset_size

    def _compute_param_loss(self, parameter_name, param, device):
        """
        Compute the EWC loss contribution for a specific parameter.
        :param parameter_name: Name of the parameter.
        :param param: Current parameter tensor.
        :param device: Device on which loss is computed.
        :return: Regularization loss for this parameter.
        """
        fisher_matrix = self.fisher_info[parameter_name].to(device)
        prev_param = self.prev_params[parameter_name].to(device)
        return (fisher_matrix * (param - prev_param) ** 2).sum()#%% md

## 4. CL: Reservoir sampling for rehearsal implementation

In [5]:
class ReservoirBuffer:
    """
    Implements reservoir sampling for rehearsal in continual learning.
    """

    def __init__(self, buffer_size: int):
        """
        Initialize the reservoir buffer.
        :param buffer_size: Maximum size of the reservoir.
        """
        self.buffer_size = buffer_size
        self.reservoir = []
        self.num_samples_seen = 0  # Tracks the total number of samples seen.

    def add_sample(self, sample: tuple):
        """
        Add a new sample to the reservoir.
        If the reservoir is full, a sample is replaced with decreasing probability.
        :param sample: A data sample (e.g., input-label pair).
        """
        self.num_samples_seen += 1
        if len(self.reservoir) < self.buffer_size:
            self.reservoir.append(sample)
        else:
            self.replace_sample_in_reservoir(sample)

    def replace_sample_in_reservoir(self, sample: tuple):
        """
        Replace a random sample in the reservoir with the new sample, based on reservoir sampling logic.
        :param sample: A data sample (e.g., input-label pair).
        """
        replacement_index = random.randint(0, self.num_samples_seen - 1)
        if replacement_index < self.buffer_size:
            self.reservoir[replacement_index] = sample

    def load_buffer(self, dataloader, device="cpu"):
        """
        Load the reservoir with samples from the dataloader.
        :param dataloader: DataLoader providing the data to be added to the reservoir.
        :param device: Device to which data should be moved before adding to the reservoir.
        """
        for inputs, labels in tqdm(dataloader, desc="Loading Reservoir"):
            inputs, labels = inputs.to(device), labels.to(device)
            for input_sample, label_sample in zip(inputs, labels):
                self.add_sample((input_sample, label_sample))

    def get_replay_dataset(self) -> TensorDataset:
        """
        Return the reservoir as a PyTorch dataset.
        :return: A TensorDataset containing the sampled data.
        :raises ValueError: If the reservoir is empty.
        """
        if not self.reservoir:
            raise ValueError("The reservoir buffer is empty.")
        inputs, labels = zip(*self.reservoir)
        return TensorDataset(torch.stack(inputs), torch.tensor(labels))


class ReplayDataLoader:
    """
    Custom DataLoader for mixing task and replay samples in each batch.
    """

    def __init__(
            self,
            task_dataset,
            replay_dataset,
            batch_size: int,
            replay_ratio: float = 0.2,
            shuffle: bool = True,
            device: str = "cuda",
            num_workers: int = 0,
    ):
        """
        Initialize the ReplayDataLoader.
        :param task_dataset: Current task dataset.
        :param replay_dataset: Replay dataset.
        :param batch_size: Total batch size.
        :param replay_ratio: Proportion of replay samples in each batch.
        :param shuffle: Shuffle datasets at the start of each epoch.
        :param device: Device for tensors ('cuda' or 'cpu').
        :param num_workers: Number of worker threads for data loading.
        """
        self.task_dataset = task_dataset
        self.replay_dataset = replay_dataset
        self.batch_size = batch_size
        self.replay_ratio = replay_ratio
        self.shuffle = shuffle
        self.device = device
        self.num_workers = num_workers

        # Compute batch sizes and validate replay ratio
        self.task_samples, self.replay_samples = self._compute_batch_sizes()
        self._validate_replay_ratio()

        # Create dataloaders
        self._create_loaders()

    def _compute_batch_sizes(self):
        """Compute the number of samples for task and replay batches."""
        task_samples = int(self.batch_size * (1 - self.replay_ratio))
        replay_samples = self.batch_size - task_samples
        return task_samples, replay_samples

    def _validate_replay_ratio(self):
        """Ensure replay ratio is feasible and print a warning if not."""
        buffer_size = len(self.replay_dataset)
        task_size = len(self.task_dataset)
        max_replay_ratio = buffer_size / (buffer_size + task_size)
        if self.replay_ratio > max_replay_ratio:
            print(
                f"Warning: Replay ratio {self.replay_ratio:.2f} exceeds the feasible maximum {max_replay_ratio:.2f}. "
                "Samples may be reused in replay batches."
            )

    def _create_loaders(self):
        """Create DataLoader instances for task and replay datasets."""
        self.task_loader = DataLoader(
            self.task_dataset,
            batch_size=self.task_samples,
            shuffle=self.shuffle,
            num_workers=self.num_workers,
        )
        self.replay_loader = DataLoader(
            self.replay_dataset,
            batch_size=self.replay_samples,
            shuffle=self.shuffle,
            num_workers=self.num_workers,
        )

    def __iter__(self):
        """Initialize dataset iterators."""
        self.task_iter = iter(self.task_loader)
        self.replay_iter = iter(self.replay_loader)
        return self

    def __next__(self):
        """Generate the next batch of mixed data."""
        try:
            task_batch = next(self.task_iter)
        except StopIteration:
            raise StopIteration

        try:
            replay_batch = next(self.replay_iter)
        except StopIteration:
            # Reset replay iterator if exhausted
            self.replay_iter = iter(self.replay_loader)
            replay_batch = next(self.replay_iter)

        task_inputs, task_labels = task_batch
        replay_inputs, replay_labels = replay_batch

        combined_inputs = torch.cat([task_inputs, replay_inputs], dim=0).to(self.device)
        combined_labels = torch.cat([task_labels, replay_labels], dim=0).to(self.device)

        return combined_inputs, combined_labels

    def __len__(self):
        """Return the number of batches."""
        return len(self.task_loader)

## 5. SNN with continual learning implementation

In [6]:
# This section of the code has been modified from:
# Martínez, F.S. (in process). Eco-efficiency of SNNs in autonomous driving (PhD. Dissertation). Universitat Oberta de Catalunya.
# Modifications include:
# - Implementation of Elastic Weight Consolidation (EWC).
# - Integration of a reservoir buffer for rehearsal.
# These modifications adapt the model to continual learning environments.

class SNNPilotNetCL(pl.LightningModule):
    def __init__(
            self,
            learning_rate=1e-4,
            beta=0.9,
            ewc_lambda=0.0,
            spike_grad=surrogate.fast_sigmoid(slope=25),
            threshold=0.5,
            learn_beta=True,
    ):
        super(SNNPilotNetCL, self).__init__()
        self.save_hyperparameters(ignore=["spike_grad"])

        # Hyperparameters
        self.learning_rate = learning_rate
        self.ewc_lambda = ewc_lambda  # Weight for EWC regularization

        # Common parameters for SNN Leaky layers
        self.leaky_layer_params = {
            "beta": beta,
            "threshold": threshold,
            "learn_beta": learn_beta,
            "spike_grad": spike_grad,
            "init_hidden": True,
        }

        # Define the EWC object
        self.ewc = EWC(self.to(DEVICE))

        # Build the spiking neural network layers
        self.net = self._build_network()

        # Loss function
        self.loss_fn = nn.MSELoss()

    def _build_network(self):
        """Construct the spiking neural network."""
        return nn.Sequential(
            nn.Conv2d(1, 24, kernel_size=5, stride=2),
            snn.Leaky(**self.leaky_layer_params),
            nn.Conv2d(24, 36, kernel_size=5, stride=2),
            snn.Leaky(**self.leaky_layer_params),
            nn.Conv2d(36, 48, kernel_size=5, stride=2),
            snn.Leaky(**self.leaky_layer_params),
            nn.Conv2d(48, 64, kernel_size=3),
            snn.Leaky(**self.leaky_layer_params),
            nn.Conv2d(64, 64, kernel_size=3),
            snn.Leaky(**self.leaky_layer_params),
            nn.Flatten(),
            nn.Linear(1152, 100),
            snn.Leaky(**self.leaky_layer_params),
            nn.Linear(100, 50),
            snn.Leaky(**self.leaky_layer_params),
            nn.Linear(50, 10),
            snn.Leaky(**self.leaky_layer_params),
            nn.Linear(10, 1),
            snn.Leaky(
                **{**self.leaky_layer_params, "output": True, "reset_mechanism": "none"}
            ),
        )

    def forward(self, x):
        """Forward pass for SNN with reset at each call."""
        utils.reset(self.net)
        membrane_potentials = []  # Record membrane potentials

        # Add a dummy time dimension if missing (for delta encoding)
        if x.dim() < 5:
            x = x.unsqueeze(1)

        # Process input through time steps
        for step in range(x.size(1)):
            step_data = x[:, step, :, :, :]
            _, mem_out = self.net(step_data)
            membrane_potentials.append(mem_out)

        return mem_out  # Output from the final time step

    def get_ewc_lambda(self):
        """Return the current EWC lambda."""
        return self.ewc_lambda

    def set_ewc_lambda(self, ewc_lambda):
        """Set a new value for EWC lambda."""
        self.ewc_lambda = ewc_lambda

    def set_ewc(self, ewc):
        """Set the EWC object for continual learning."""
        self.ewc = ewc

    def training_step(self, batch, batch_idx):
        """Perform a training step."""
        images, labels = batch
        predictions = self(images)
        labels = labels.unsqueeze(1)
        task_loss = self.loss_fn(predictions, labels)

        # Compute EWC regularization loss
        ewc_loss = (
            self.ewc_lambda * self.ewc.ewc_loss() if self.ewc_lambda != 0 else 0.0
        )
        total_loss = task_loss + ewc_loss

        # Log losses
        self.log("ewc_loss", ewc_loss)
        self.log("train_loss", total_loss)
        return total_loss

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        """Perform a validation step."""
        images, labels = batch
        predictions = self(images)
        labels = labels.unsqueeze(1)
        val_loss = self.loss_fn(predictions, labels)
        self.log("val_loss", val_loss)
        return val_loss

    def test_step(self, batch, batch_idx, dataloader_idx=0):
        """Perform a test step."""
        images, labels = batch
        predictions = self(images)
        labels = labels.unsqueeze(1)
        test_loss = self.loss_fn(predictions, labels)
        self.log("test_loss", test_loss)
        return test_loss

    def configure_optimizers(self):
        """Configure the optimizer."""
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)


class H5SpikingDataset(Dataset):
    def __init__(self, h5_file_path, split="train", use_test_split=False):
        self.h5_file_path = h5_file_path
        self.split = split
        self.use_test_split = use_test_split

        # Open the H5 file in read mode
        self.h5_file = h5py.File(self.h5_file_path, "r")

        # Load images and labels based on split
        self.images, self.labels = self._get_images_and_labels()

    def _get_images_and_labels(self):
        """Helper method to retrieve images and labels based on the dataset split."""
        dataset_mapping = {
            "train": ("train_images", "train_labels"),
            "val": ("val_images", "val_labels"),
            "test": (
                "test_images" if self.use_test_split else "val_images",
                "test_labels" if self.use_test_split else "val_labels",
            ),
        }

        image_key, label_key = dataset_mapping[self.split]
        return self.h5_file[image_key], self.h5_file[label_key]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Retrieve the encoded image and label
        image = torch.tensor(self.images[idx])  # Convert to PyTorch tensor
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label

    def close(self):
        self.h5_file.close()


class H5PilotNetDataModule(pl.LightningDataModule):
    SPLIT_TRAIN = "train"
    SPLIT_VAL = "val"
    SPLIT_TEST = "test"

    def __init__(
            self,
            h5_file_path,
            train_on_task: int = 0,
            batch_size: int = 64,
            concat: bool = False,
            val_all: bool = True,
            replay: bool = False,
            reservoir_buffer=None,
            replay_ratio: float = 0.2,
            num_workers: int = 0,
    ):
        """
        Initializes the H5PilotNetDataModule for training, validation, and testing.

        :param h5_file_path: Path(s) to the H5 dataset(s). Can be a single path or a list of paths.
        :param train_on_task: Index of the dataset to use for training (ignored if `concat=True`).
        :param batch_size: Batch size for training, validation, and testing.
        :param concat: Whether to concatenate datasets across tasks for training and validation.
        :param val_all: Whether to validate/test on all datasets or only the `train_on_task` dataset.
        :param replay: Whether to activate replay mode for training using reservoir sampling.
        :param reservoir_buffer: Replay buffer instance for replay training.
        :param replay_ratio: Ratio of replay samples to task samples in each training batch.
        :param num_workers: Number of workers for data loading.
        """
        super().__init__()
        self.batch_size = batch_size
        self.reservoir_buffer = reservoir_buffer
        self.replay_ratio = replay_ratio
        self.replay = replay
        self.concat = concat
        self.train_on_task = 0 if concat else train_on_task
        self.val_all = val_all
        self.num_workers = num_workers

        # Ensure `h5_file_path` is a list, even if a single dataset is provided
        self.h5_file_paths = (
            [h5_file_path] if not isinstance(h5_file_path, list) else h5_file_path
        )

        # Issue a warning if the number of datasets is insufficient
        if len(self.h5_file_paths) < 2 and (train_on_task != 0 or val_all or concat):
            print("Warning: Not enough datasets provided. Using single dataset.")

    def set_reservoir_buffer(self, reservoir_buffer):
        """
        Sets the reservoir buffer for replay training.

        :param reservoir_buffer: Instance of a ReservoirBuffer.
        """
        self.reservoir_buffer = reservoir_buffer

    def activate_replay(self, replay=True):
        """
        Activates or deactivates replay mode depending on the state of the reservoir buffer.

        :param replay: Set to True to activate replay mode. Defaults to True.
        """
        if self.reservoir_buffer is None and replay:
            print("There's no buffer to activate replay. Turning off replay.")
            self.replay = False
        else:
            self.replay = replay

    def setup(self, stage=None):
        """
        Prepares datasets for training, validation, and testing.

        :param stage: Optional stage specification (e.g., "fit" or "test"). Ignored here.
        """
        # Construct validation and test datasets
        self.val_datasets = self._build_validation_datasets()
        self.test_datasets = self._build_test_datasets()

        # Construct training dataset
        if self.concat:
            # Concatenate datasets across tasks if `concat` is True
            self.train_datasets_to_close = [
                self._build_train_dataset(file) for file in self.h5_file_paths
            ]
            self.train_dataset = ConcatDataset(self.train_datasets_to_close)
            # Add concatenated validation dataset for compatibility
            self.val_datasets.append(ConcatDataset(self.val_datasets))
        else:
            # Use only the specified task for training
            self.train_dataset = self._build_train_dataset(
                self.h5_file_paths[self.train_on_task]
            )

    def teardown(self, stage=None):
        """
        Cleans up resources and closes datasets after the training/testing phase.

        :param stage: Optional stage specification (e.g., "fit" or "test"). Ignored here.
        """
        if self.concat:
            # Close all training datasets if `concat` was used
            for dataset in self.train_datasets_to_close:
                self._close_single_dataset(dataset)
            # Avoid double-closing concatenated validation datasets
            self.val_datasets.pop()
        else:
            # Close the single specified training dataset
            self._close_single_dataset(self.train_dataset)

        # Close all validation and test datasets
        for dataset in self.val_datasets:
            self._close_single_dataset(dataset)
        for dataset in self.test_datasets:
            self._close_single_dataset(dataset)

    def train_dataloader(self):
        """
        Returns the dataloader for training.
        If replay is enabled, a ReplayDataLoader is used to mix task and replay samples.

        :return: Dataloader for training.
        """
        if self.replay:
            return ReplayDataLoader(
                task_dataset=self.train_dataset,
                replay_dataset=self.reservoir_buffer.get_replay_dataset(),
                batch_size=self.batch_size,
                replay_ratio=self.replay_ratio,
                shuffle=True,
                num_workers=self.num_workers,
            )
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        """
        Returns the dataloaders for validation.

        :return: A single dataloader or a list of dataloaders, depending on `val_all`.
        """
        if self.val_all:
            return [
                DataLoader(
                    dataset,
                    batch_size=self.batch_size,
                    shuffle=False,
                    num_workers=self.num_workers,
                )
                for dataset in self.val_datasets
            ]
        return DataLoader(
            self.val_datasets[self.train_on_task],
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        """
        Returns the dataloaders for testing.

        :return: A single dataloader or a list of dataloaders, depending on `val_all`.
        """
        if self.val_all:
            return [
                DataLoader(
                    dataset,
                    batch_size=self.batch_size,
                    shuffle=False,
                    num_workers=self.num_workers,
                )
                for dataset in self.test_datasets
            ]
        return DataLoader(
            self.test_datasets[self.train_on_task],
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def _build_train_dataset(self, file):
        """
        Builds a single training dataset for the given file.

        :param file: Path to the H5 dataset file.
        :return: An instance of H5SpikingDataset for the training split.
        """
        return H5SpikingDataset(file, split=self.SPLIT_TRAIN)

    def _build_validation_datasets(self):
        """
        Creates validation datasets for all provided files.

        :return: A list of H5SpikingDataset instances for validation.
        """
        return [
            H5SpikingDataset(file, split=self.SPLIT_VAL) for file in self.h5_file_paths
        ]

    def _build_test_datasets(self):
        """
        Creates test datasets for all provided files.

        :return: A list of H5SpikingDataset instances for testing.
        """
        return [
            H5SpikingDataset(file, split=self.SPLIT_TEST) for file in self.h5_file_paths
        ]

    def _close_single_dataset(self, dataset):
        """
        Closes a single dataset, releasing its resources.

        :param dataset: Instance of H5SpikingDataset to be closed.
        """
        dataset.close()

## 6. Data management setup

In [7]:
# Constants for paths and filenames
BASE_DIR = "SNN/data/encoded"
DATASET_FILENAME = "encoded_dataset_rate_numsteps_25_gain_0.5.h5"

# Function to create track file paths
def create_track_path(track, encoding, dataset_filename):
    return os.path.join(BASE_DIR, track, encoding, "Encoded_Datasets", dataset_filename)

# Selected encoding and tracks
tracks = ["track1", "track2"]
selected_encoding = "rate"

# Generate paths for tracks using a loop
track_paths = [
    create_track_path(track, selected_encoding, DATASET_FILENAME) for track in tracks
]

# Combined DataModule creation (paths list passed as an argument)
data_module = H5PilotNetDataModule(
    h5_file_path=track_paths, batch_size=64, num_workers=3
)

# Output directories
timestamp = datetime.now().strftime("%d%H%M%S")
run_dir = os.path.join("SNN/CL", timestamp)
os.makedirs(run_dir, exist_ok=True)

In [8]:
# Define constants
BACKUP_DIR = "/content/drive/MyDrive/TFM/SNN/CL/running/bkp/"

if is_colab:
    # Ensure backup folder exists
    def create_folder_if_not_exists(folder_path):
        """Ensures the specified folder exists or creates it."""
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    create_folder_if_not_exists(BACKUP_DIR)

    def build_rsync_command(output_folder, drive_folder, backup_folder):
        """
        Builds the rsync command to sync the output folder to Google Drive,
        keeping backups of deleted files.
        """
        return f"rsync -av --delete --backup --backup-dir={backup_folder} --progress {output_folder} {drive_folder}"

    # Definition of folders
    output_folder = run_dir
    drive_folder = f"/content/drive/MyDrive/TFM/SNN/CL/running/{timeref}"
    rsync_command = build_rsync_command(output_folder, drive_folder, BACKUP_DIR)

    def sync_data_with_backup():
        """
        Runs rsync to sync data, backs up deleted files,
        and handles Google Drive unmount/remount operations.
        """
        # Sync output folder to drive folder
        timestamp = datetime.now().strftime("%d%H%M%S")
        print(
            f"Syncing {output_folder} to {drive_folder + timestamp} and deleting outdated files..."
        )
        sync_process = Popen(rsync_command, shell=True)
        sync_process.wait()  # Wait for completion

        # Flush and unmount Google Drive
        print("Flushing and unmounting Google Drive...")
        drive.flush_and_unmount()

        # Remount Google Drive
        print("Remounting Google Drive...")
        drive.mount("/content/drive")

##  7. Model Training and Evaluation Setup

In [9]:
# Enforce results reproducibility
# Set a fixed seed value
SEED = 42

# Select device dynamically
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Select device dynamically
print(f"Using device: {DEVICE}")

# Select models to run - Toggle True/False to train the models
loss_before_fit = False # Get loss on task1/2 before any training is done
retrain_task1 = False  # Retrain model on task1 or load latest best_model from best_model_path
TASK1_NUM_RUNS = 5
compute_FIM = False # Wheter to compute FIM or reuse a saved one

# task2 variants
task2_naive = False
task2_cumulative = False
task2_ecw = False
task2_rehearsal = True
TASK2_NUM_RUNS = 2

# Training epochs
EPOCHS = 14  # Number of training epochs
BATCH_LIMIT = 3 #1.0  # limit batches for testing

# Define task1 best model path
if is_colab:
    best_model_path = {
        'task1': '/content/drive/MyDrive/TFM/SNN/CL/18213435/SNN_CL_task1_rate_numsteps_25_gain_0.5/version_0/checkpoints/epoch=26-step=2079.ckpt'}
else:
    best_model_path = {
        'task1': 'SNN/CL/18213435/SNN_CL_task1_rate_numsteps_25_gain_0.5/version_0/checkpoints/epoch=26-step=2079.ckpt'}
results = {}  # Dictionary to store results

Using device: cuda


## 8. Train on task 1: Base model

In [10]:
if retrain_task1:
    # Seed
    seed_everything(SEED, workers=True)

    # Train on task1
    task_name = 'task1'
    for run in range(TASK1_NUM_RUNS):
        run_name = 'run' + str(run)

        # Create model
        model = SNNPilotNetCL(learning_rate=1e-4, beta=0.9, ewc_lambda=0.0)

        # Save initialization weights for analysis if needed
        torch.save(model.state_dict(), output_dir + '/init_weights.pth')

        # Create a logger for TensorBoard and CSV
        logger_name = f"SNN_CL_{task_name_{track_paths[0].split('_', 3)[-1]}"[:-3]
        logger_TSB = TensorBoardLogger(run_dir, name=logger_name)
        logger_CSV = CSVLogger(run_dir, name=logger_name, version=logger_TSB.version)
        logger = [logger_TSB, logger_CSV]

        # Output_dir
        output_dir = os.path.join(run_dir, logger_name, 'version_' + str(logger_TSB.version))
        os.makedirs(output_dir, exist_ok=True)



        # Checkpoint callbacks
        checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_loss/dataloader_idx_0")  # default best model
        checkpoint_callback_last = ModelCheckpoint(save_last=True)  # default best model

        # CodeCarbon tracker and Callback
        tracker = EmissionsTracker(
            project_name=logger_name,
            output_dir=output_dir,
            output_file='emissions_' + logger_name + '.csv',
            log_level="error")

        co2_callback = CodeCarbonCallback(tracker)

        # Trainer configuration
        trainer = pl.Trainer(
            max_epochs=EPOCHS,
            accelerator='cuda',
            devices='auto',
            precision='16-mixed',
            logger=logger,
            limit_train_batches=BATCH_LIMIT,
            log_every_n_steps=50,
            callbacks=[checkpoint_callback,
                       checkpoint_callback_last,
                       MyProgressBar(),
                       co2_callback]
        )

        if loss_before_fit:
            # Get loss before fit
            model.eval()
            test1, test2 = trainer.test(model, datamodule=data_module)
            results.setdefault('no_train', {}).setdefault('track1_test_loss', []).append(
                test1['test_loss/dataloader_idx_0'])
            results.setdefault('no_train', {}).setdefault('track2_test_loss', []).append(
                test2['test_loss/dataloader_idx_1'])

        # Train the model on first track
        model.train()
        trainer.fit(model, datamodule=data_module)

        # Set best model  for task1
        best_model = checkpoint_callback.best_model_path

        # Evaluate model on first task
        model.eval()
        test1, test2 = trainer.test(model, ckpt_path=best_model, datamodule=data_module)

        results.setdefault('task1', {}).setdefault('track1_test_loss', []).append(test1['test_loss/dataloader_idx_0'])
        results.setdefault('task1', {}).setdefault('track2_test_loss', []).append(test2['test_loss/dataloader_idx_1'])
        results.setdefault('task1', {}).setdefault('best_model_path', []).append(best_model)

        # Save best model path
        # Find the index of the minimum value in track1_test_loss
        min_index = results['task1']['track1_test_loss'].index(min(results['task1']['track1_test_loss']))

        # Retrieve the corresponding best_model_path
        best_model_path['task1'] = results['task1']['best_model_path'][min_index]

        data_module.teardown()

        if is_colab:
            sync_data_with_backup()

## 9. Train on task2: Naive model

In [11]:
if task2_naive:
    # Seed
    seed_everything(SEED, workers=True)

    # Train on task2 naive
    task_name = 'task2_naive'
    for run in range(TASK2_NUM_RUNS):

        # Load model and FIM data
        model = SNNPilotNetCL.load_from_checkpoint(best_model_path['task1'],
                                                     spike_grad=surrogate.fast_sigmoid(slope=25))

        # Train
        model.train()

        # Checkpoint callbacks
        checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_loss/dataloader_idx_1")  # default best model
        checkpoint_callback_last = ModelCheckpoint(save_last=True)  # default best model

        # Reset logger for TensorBoard and CSV
        logger_name = f"SNN_CL_{task_name}_{track_paths[0].split('_', 3)[-1]}"[:-3]
        logger_TSB = TensorBoardLogger(run_dir, name=logger_name)
        logger_CSV = CSVLogger(run_dir, name=logger_name, version=logger_TSB.version)
        logger = [logger_TSB, logger_CSV]

        # Output_dir
        output_dir = os.path.join(run_dir, logger_name, 'version_' + str(logger_TSB.version))
        os.makedirs(output_dir, exist_ok=True)

        # CodeCarbon tracker and Callback
        tracker = EmissionsTracker(
            project_name=logger_name,
            output_dir=output_dir,
            output_file='emissions_' + logger_name + '.csv',
            log_level="error")

        co2_callback = CodeCarbonCallback(tracker)

        # Instantiate new trainer to increase max_epochs and reset callbacks
        trainer = pl.Trainer(
            max_epochs=EPOCHS * 2,
            accelerator='cuda',
            devices='auto',
            precision='16-mixed',
            logger=logger,
            limit_train_batches=BATCH_LIMIT,
            log_every_n_steps=50,
            callbacks=[checkpoint_callback,
                       checkpoint_callback_last,
                       MyProgressBar(),
                       co2_callback])

        # Setup datamodule for current task
        data_module.train_on_task = 1

        trainer.fit(model, ckpt_path=best_model_path['task1'], datamodule=data_module)

        # Set best model for this run
        best_model = checkpoint_callback.best_model_path

        # Evaluate model on second task
        model.eval()
        test1, test2 = trainer.test(model, ckpt_path=best_model, datamodule=data_module)

        results.setdefault(task_name, {}).setdefault('track1_test_loss', []).append(test1['test_loss/dataloader_idx_0'])
        results.setdefault(task_name, {}).setdefault('track2_test_loss', []).append(test2['test_loss/dataloader_idx_1'])
        results.setdefault(task_name, {}).setdefault('best_model_path', []).append(best_model)

        data_module.teardown()

        if is_colab:
            sync_data_with_backup()

## 10. Train on task2: Cumulative model

In [12]:
if task2_cumulative:
    # Seed
    seed_everything(SEED, workers=True)

    # Train on task2 cumulative
    task_name = 'task2_cumulative'
    for run in range(TASK2_NUM_RUNS):

        # Load model and FIM data
        model = SNNPilotNetCL.load_from_checkpoint(best_model_path['task1'],
                                                     spike_grad=surrogate.fast_sigmoid(slope=25))

        # Train
        model.train()

        # Checkpoint callbacks
        checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_loss/dataloader_idx_1")  # default best model
        checkpoint_callback_last = ModelCheckpoint(save_last=True)  # default best model

        # Reset logger for TensorBoard and CSV
        logger_name = f"SNN_CL_{task_name}_{track_paths[0].split('_', 3)[-1]}"[:-3]
        logger_TSB = TensorBoardLogger(run_dir, name=logger_name)
        logger_CSV = CSVLogger(run_dir, name=logger_name, version=logger_TSB.version)
        logger = [logger_TSB, logger_CSV]

        # Output_dir
        output_dir = os.path.join(run_dir, logger_name, 'version_' + str(logger_TSB.version))
        os.makedirs(output_dir, exist_ok=True)

        # CodeCarbon tracker and Callback
        tracker = EmissionsTracker(
            project_name=logger_name,
            output_dir=output_dir,  # change in all
            output_file='emissions_' + logger_name + '.csv',
            log_level="error")

        co2_callback = CodeCarbonCallback(tracker)

        # Instantiate new trainer to increase max_epochs and reset callbacks
        trainer = pl.Trainer(
            max_epochs=EPOCHS * 2,
            accelerator='cuda',
            devices='auto',
            precision='16-mixed',
            logger=logger,
            limit_train_batches=BATCH_LIMIT,
            log_every_n_steps=50,
            callbacks=[checkpoint_callback,
                       checkpoint_callback_last,
                       MyProgressBar(),
                       co2_callback])

        # Setup datamodule for current task
        data_module.concat = True

        trainer.fit(model, ckpt_path=best_model_path['task1'], datamodule=data_module)

        # Set best model for this run
        best_model = checkpoint_callback.best_model_path

        # Evaluate model on second task
        model.eval()
        test1, test2 = trainer.test(model, ckpt_path=best_model, datamodule=data_module)

        results.setdefault(task_name, {}).setdefault('track1_test_loss', []).append(test1['test_loss/dataloader_idx_0'])
        results.setdefault(task_name, {}).setdefault('track2_test_loss', []).append(test2['test_loss/dataloader_idx_1'])
        results.setdefault(task_name, {}).setdefault('best_model_path', []).append(best_model)

        data_module.teardown()

        if is_colab:
            sync_data_with_backup()

## 11. Train on task2: ECW models

In [13]:
if task2_ecw:
    # Generate lambdas 
    # steps = np.logspace(1, 15, num=5, base=10)
    steps = [1e9, 1e10, 5e10, 1e11, 1e12]
    lambdas = np.round(steps / 10 ** np.floor(np.log10(steps)), 1) * 10 ** np.floor(np.log10(steps))
    lambdas = {f"{value:.1e}": value for value in lambdas}

    if compute_FIM:
        # Seed
        seed_everything(SEED, workers=True)
        for run in range(TASK2_NUM_RUNS):
            # Load task1 best model
            model = SNNPilotNetCL.load_from_checkpoint(best_model_path['task1'],
                                                         spike_grad=surrogate.fast_sigmoid(slope=25))
            # Initialize EWC after first task
            model.ewc.save_prev_params()

            # Compute FIM diagonal
            data_module.setup()
            model.ewc.compute_fisher(data_module.train_dataloader(), device=DEVICE)

            # Save EWC data
            ewc_data = {
                "prev_params": model.ewc.prev_params,
                "fisher_info": model.ewc.fisher_info,
            }

            best_model_path_FIM = f"{best_model_path['task1'][:-5]}_{run}.FIM"
            torch.save(ewc_data, best_model_path_FIM)

            data_module.teardown()

            if is_colab:
                sync_data_with_backup()

    for name, l in lambdas.items():
        # Seed
        seed_everything(SEED, workers=True)

        task_name = f"task2_ewc_{name}"
        print("*** Start training on " + task_name)

        for run in range(TASK2_NUM_RUNS):
            # Load task1 best model
            model = SNNPilotNetCL.load_from_checkpoint(best_model_path['task1'],
                                                         spike_grad=surrogate.fast_sigmoid(slope=25))

            # Load FIM data
            ewc_data = torch.load(f"{best_model_path['task1'][:-5]}_{run}.FIM", weights_only=False)
            model.ewc.prev_params = ewc_data['prev_params']
            model.ewc.fisher_info = ewc_data['fisher_info']

            # Set lambda value
            model.ewc_lambda = l

            # Train
            model.train()

            # Checkpoint callbacks
            checkpoint_callback = ModelCheckpoint(save_top_k=1,
                                                  monitor="val_loss/dataloader_idx_1")  # default best model
            checkpoint_callback_last = ModelCheckpoint(save_last=True)  # default best model

            # Reset logger for TensorBoard and CSV
            logger_name = f"SNN_CL_{task_name}_{track_paths[0].split('_', 3)[-1]}"[:-3]
            logger_TSB = TensorBoardLogger(run_dir, name=logger_name)
            logger_CSV = CSVLogger(run_dir, name=logger_name, version=logger_TSB.version)
            logger = [logger_TSB, logger_CSV]

            # Output_dir
            output_dir = os.path.join(run_dir, logger_name, 'version_' + str(logger_TSB.version))
            os.makedirs(output_dir, exist_ok=True)

            # CodeCarbon tracker and Callback
            tracker = EmissionsTracker(
                project_name=logger_name,
                output_dir=output_dir,  # change in all
                output_file='emissions_' + logger_name + '.csv',
                log_level="error")

            co2_callback = CodeCarbonCallback(tracker)

            # Instantiate new trainer to increase max_epochs and reset callbacks
            trainer = pl.Trainer(
                max_epochs=EPOCHS * 2,
                accelerator='cuda',
                devices='auto',
                precision='16-mixed',
                logger=logger,
                limit_train_batches=BATCH_LIMIT,
                log_every_n_steps=50,
                callbacks=[checkpoint_callback,
                           checkpoint_callback_last,
                           MyProgressBar(),
                           co2_callback])

            # Setup datamodule for current task
            data_module.concat = False
            data_module.train_on_task = 1

            trainer.fit(model, ckpt_path=best_model_path['task1'], datamodule=data_module)

            # Set best model for this run
            best_model = checkpoint_callback.best_model_path

            # Evaluate model on second task
            model.eval()
            test1, test2 = trainer.test(model, ckpt_path=best_model, datamodule=data_module)

            results.setdefault(task_name, {}).setdefault('track1_test_loss', []).append(
                test1['test_loss/dataloader_idx_0'])
            results.setdefault(task_name, {}).setdefault('track2_test_loss', []).append(
                test2['test_loss/dataloader_idx_1'])
            results.setdefault(task_name, {}).setdefault('best_model_path', []).append(best_model)
            data_module.teardown()

            if is_colab:
                sync_data_with_backup()

## 12. Train on task2: REHEARSAL models

In [14]:
if task2_rehearsal:
    # Define replay ratios
    replay_ratios = [0.05, 0.1, 0.15, 0.2, 0.25]

    for i, r in enumerate(replay_ratios):
        task_name = f"task2_Rep{r}"
        print("*** Start training on " + task_name)

        # Seed sequence
        seed_everything(SEED, workers=True)

        for run in range(TASK2_NUM_RUNS):
            # Load task1 model
            model = SNNPilotNetCL.load_from_checkpoint(best_model_path['task1'],
                                                         spike_grad=surrogate.fast_sigmoid(slope=25))

            # Create and fill reservoir buffer
            reservoir = ReservoirBuffer(buffer_size=1200)  # Greater that 1232 will ensure up to 25% rratio

            # Configure data module
            data_module.train_on_task = 0
            data_module.setup()
            reservoir.load_buffer(data_module.train_dataloader())
            data_module.teardown()

            # Train
            model.train()

            # Checkpoint callbacks
            checkpoint_callback = ModelCheckpoint(save_top_k=1,
                                                  monitor="val_loss/dataloader_idx_1")  # default best model
            checkpoint_callback_last = ModelCheckpoint(save_last=True)  # default best model

            # Reset logger for TensorBoard and CSV
            logger_name = f"SNN_CL_{task_name}_{track_paths[0].split('_', 3)[-1]}"[:-3]
            logger_TSB = TensorBoardLogger(run_dir, name=logger_name)
            logger_CSV = CSVLogger(run_dir, name=logger_name, version=logger_TSB.version)
            logger = [logger_TSB, logger_CSV]

            # Output_dir
            output_dir = os.path.join(run_dir, logger_name, 'version_' + str(logger_TSB.version))
            os.makedirs(output_dir, exist_ok=True)

            # CodeCarbon tracker and Callback
            tracker = EmissionsTracker(
                project_name=logger_name,
                output_dir=output_dir,  # change in all
                output_file='emissions_' + logger_name + '.csv',
                log_level="error")

            co2_callback = CodeCarbonCallback(tracker)

            # Instantiate new trainer to increase max_epochs and reset callbacks
            trainer = pl.Trainer(
                max_epochs=EPOCHS * 2,
                accelerator='cuda',
                devices='auto',
                precision='16-mixed',
                logger=logger,
                limit_train_batches=BATCH_LIMIT,
                log_every_n_steps=50,
                callbacks=[checkpoint_callback,
                           checkpoint_callback_last,
                           MyProgressBar(),
                           co2_callback])

            # Setup datamodule for current task
            data_module.concat = False
            data_module.train_on_task = 1

            # Load buffer on module and activate replay
            data_module.set_reservoir_buffer(reservoir)
            data_module.activate_replay()
            data_module.replay_ratio = r

            trainer.fit(model, ckpt_path=best_model_path['task1'], datamodule=data_module)

            # Set best model for this run
            best_model = checkpoint_callback.best_model_path

            # Evaluate model on second task
            model.eval()
            test1, test2 = trainer.test(model, ckpt_path=best_model, datamodule=data_module)

            results.setdefault(task_name, {}).setdefault('track1_test_loss', []).append(
                test1['test_loss/dataloader_idx_0'])
            results.setdefault(task_name, {}).setdefault('track2_test_loss', []).append(
                test2['test_loss/dataloader_idx_1'])
            results.setdefault(task_name, {}).setdefault('best_model_path', []).append(best_model)

            data_module.teardown()

            if is_colab:
                sync_data_with_backup()

Seed set to 42


*** Start training on task2_Rep0.05


Loading Reservoir: 100%|██████████| 77/77 [00:14<00:00,  5.17it/s]
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/scosta/miniconda3/envs/tfmenv/lib/python3.12/site-packages/lightning_fabric/loggers/csv_logs.py:268: Experiment logs directory SNN/CL/04171818/SNN_CL_task2_Rep0.05_rate_numsteps_25_gain_0.5/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
Restoring states from the checkpoint path at SNN/CL/18213435/SNN_CL_task1_rate_numsteps_25_gain_0.5/version_0/checkpoints/epoch=26-step=2079.ckpt
/home/scosta/miniconda3/envs/tfmenv/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:273: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["ModelCheckpoint{'monitor': 'val_loss/datal

Epoch 27: 100%|██████████| 3/3 [00:24<00:00,  0.12it/s, v_num=0]

/home/scosta/miniconda3/envs/tfmenv/lib/python3.12/site-packages/codecarbon/output.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])
`Trainer.fit` stopped: `max_epochs=28` reached.


...

Epoch 27: 100%|██████████| 3/3 [00:35<00:00,  0.08it/s, v_num=1]

/home/scosta/miniconda3/envs/tfmenv/lib/python3.12/site-packages/codecarbon/output.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])
`Trainer.fit` stopped: `max_epochs=28` reached.


Epoch 27: 100%|██████████| 3/3 [00:35<00:00,  0.08it/s, v_num=1]


Restoring states from the checkpoint path at SNN/CL/04171818/SNN_CL_task2_Rep0.25_rate_numsteps_25_gain_0.5/version_1/checkpoints/epoch=27-step=2082.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at SNN/CL/04171818/SNN_CL_task2_Rep0.25_rate_numsteps_25_gain_0.5/version_1/checkpoints/epoch=27-step=2082.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0             DataLoader 1
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.025967128574848175      0.18998223543167114
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [15]:
# Convert the dictionary to a DataFrame for a nice tabular display
df = pd.DataFrame(results).T

# Display the DataFrame
df

,track1_test_loss,track2_test_loss,best_model_path
task2_Rep0.05,"[0.027404872700572014, 0.02574142999947071]","[0.18060673773288727, 0.1745484322309494]",[SNN/CL/04171818/SNN_CL_task2_Rep0.05_rate_num...
task2_Rep0.1,"[0.0256203543394804, 0.02879573032259941]","[0.18278616666793823, 0.19607193768024445]",[SNN/CL/04171818/SNN_CL_task2_Rep0.1_rate_nums...
task2_Rep0.15,"[0.02579297125339508, 0.030834665521979332]","[0.18397797644138336, 0.20187553763389587]",[SNN/CL/04171818/SNN_CL_task2_Rep0.15_rate_num...
task2_Rep0.2,"[0.026331111788749695, 0.026042047888040543]","[0.18933089077472687, 0.17524360120296478]",[SNN/CL/04171818/SNN_CL_task2_Rep0.2_rate_nums...
task2_Rep0.25,"[0.026297450065612793, 0.025967128574848175]","[0.17814020812511444, 0.18998223543167114]",[SNN/CL/04171818/SNN_CL_task2_Rep0.25_rate_num...
